In [134]:
import re
import os


def extract_data_from_txt(file_path):
    """
    Extract bounding box parameters, label, image name, and probability from a given .txt file.
    
    Parameters:
    - file_path: path to the .txt file.
    
    Returns:
    - List of tuples with format (img_name, x_min, y_min, x_max, y_max, label, image_dimensions, prob).
    """
    
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    # Extract image name from the file path
    img_name = os.path.basename(file_path).replace("2021", "").split("_")[0]
    
    # List to store extracted data
    data = []
    h, w = map(int, lines[-1].strip().split(": ")[1].split(", "))
    for line in lines:
        if "BBOX" in line:
            # Extract bbox coordinates
            bbox = re.search(r"tensor\(\[(.+?)\]\)", line).group(1).split(", ")
            x_min, y_min, x_max, y_max = map(float, bbox)
            
            # Extract label
            label = re.search(r"LABEL: (.+?) \(ID", line).group(1)
            
            # Extract probability
            prob = float(re.search(r"PROB: (.+)", line).group(1))
            
            data.append((str(img_name), x_min, y_min, x_max, y_max, label, prob, f"({h}|{w})"))
    
    return data




In [144]:
import os

directory_path = "output/from_dataset/filtered_unknown/SOWODB_backup/"
desired_suffix = "t1.pth.txt"

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter the files based on the desired suffix
filtered_files = [f for f in all_files if f.endswith(desired_suffix)]

print(len(filtered_files))


4952


In [145]:
# List to store all the extracted data from the files
all_data = []

# Extract data from all the files
for path in filtered_files:
    all_data.extend(extract_data_from_txt(directory_path + path))

# Convert the extracted data into a CSV format
csv_output_path = "bbox_data_unscaled.csv"

# Sort all_data based on the image_id before writing to the CSV file
all_data.sort(key=lambda x: int(x[0]))

# Writing the combined data to a CSV file
with open(csv_output_path, "w") as csv_file:
    # Writing the header
    csv_file.write("img_name,x_min,y_min,x_max,y_max,label,prob,scaled_size\n")
    
    # Writing the data
    for row in all_data:
        csv_file.write(",".join(map(str, row)) + "\n")

# 
csv_output_path


'bbox_data_unscaled.csv'

In [146]:
import pandas as pd
import os
from PIL import Image

def get_actual_dimensions(image_id, image_dir):
    """
    Get the actual dimensions (width, height) of an image given its image_id.

    Parameters:
    - image_id: ID/name of the image (without file extension).
    - image_dir: Directory containing the images.

    Returns:
    - Tuple (width, height) of the actual image dimensions.
    """
    image_path = os.path.join(image_dir, image_id + ".jpg")
    
    # Check if the image exists, if not, drop the first 2 digits and try again
    if not os.path.exists(image_path):
        image_id = image_id[2:]
        image_path = os.path.join(image_dir, image_id + ".jpg")
    
    with Image.open(image_path) as img:
        return img.size

def scale_bbox(row, image_dir):
    # Get actual image dimensions
    actual_width, actual_height = get_actual_dimensions(row['img_name'], image_dir)

    # Store the actual size as a tuple-style string
    row['actual_size'] = f"({actual_height}|{actual_width})"
    
    # Get scaled dimensions from the CSV
    scaled_height, scaled_width = tuple(map(int, row['scaled_size'][1:-1].split('|')))
    
    # Compute scaling factors
    x_scale = actual_width / scaled_width
    y_scale = actual_height / scaled_height
    
    # Scale bounding box coordinates
    row['x_min'] *= x_scale
    row['y_min'] *= y_scale
    row['x_max'] *= x_scale
    row['y_max'] *= y_scale
    
    return row

image_dir = "/training_data_2/yuetian/OWOD/JPEGImages"
scaled_data =  pd.read_csv('bbox_data_unscaled.csv', dtype={'img_name': str})

# Apply the scaling function to each row of the DataFrame
scaled_data = scaled_data.apply(lambda row: scale_bbox(row, image_dir), axis=1)
scaled_data.to_csv('bbox_data_scaled.csv', index=False)
scaled_data.head()


,img_name,x_min,y_min,x_max,y_max,label,prob,scaled_size,actual_size
0,00000018,29.118519,169.901154,84.446719,276.514942,unknown,0.919638,(750|1333),(285|380)
1,00000018,86.850668,50.791028,160.615367,158.934734,unknown,0.893323,(750|1333),(285|380)
2,00000018,30.860504,233.492178,58.715473,277.912354,unknown,0.889401,(750|1333),(285|380)
3,00000018,40.825769,191.754954,130.273464,286.123736,unknown,0.885862,(750|1333),(285|380)
4,00000018,28.236081,163.481472,103.817682,283.859392,person,0.876383,(750|1333),(285|380)


In [147]:
scaled_data = scaled_data[scaled_data['img_name'].str.len() >= 10]
scaled_data.to_csv('bbox_data_scaled.csv', index=False)

scaled_data

,img_name,x_min,y_min,x_max,y_max,label,prob,scaled_size,actual_size
5,000000000139,-34.309862,407.299878,82.574335,425.321062,unknown,0.997236,(800|1201),(426|640)
6,000000000139,17.275577,417.012518,525.628856,429.082962,unknown,0.997198,(800|1201),(426|640)
7,000000000139,-0.428336,76.326047,3.251317,328.314845,unknown,0.982535,(800|1201),(426|640)
8,000000000139,599.953479,372.585724,638.359980,424.054777,unknown,0.976601,(800|1201),(426|640)
9,000000000139,-0.245864,375.194175,6.168726,423.172425,unknown,0.975435,(800|1201),(426|640)
...,...,...,...,...,...,...,...,...,...
32976,000000581781,11.741699,351.056465,265.509049,478.230396,unknown,0.954600,(800|1071),(478|640)
32977,000000581781,364.439186,429.945286,452.602144,480.675784,unknown,0.952881,(800|1071),(478|640)
32978,000000581781,32.025218,-0.129598,107.172034,16.083565,unknown,0.942895,(800|1071),(478|640)
32979,000000581781,-1.228728,217.778473,17.414693,309.652941,unknown,0.942614,(800|1071),(478|640)


In [154]:
l1=['000000575970 0.093 241.1 87.0 280.9 95.9\n', '000000575970 0.085 261.2 105.2 277.0 120.5\n', '000000575970 0.080 242.6 107.0 257.5 119.2\n', '000000575970 0.078 481.0 89.3 516.1 98.0\n', '000000575970 0.077 276.7 79.8 296.6 89.3\n', '000000519569 0.088 146.7 149.6 158.3 160.3\n', '000000239627 0.098 315.0 149.2 519.5 383.5\n', '000000239627 0.079 311.4 150.4 542.6 403.7\n', '000000135670 0.126 1.1 12.9 375.1 474.7\n', '000000114770 0.199 294.5 -0.9 629.1 200.0\n', '000000114770 0.195 297.4 1.9 632.9 241.6\n', '000000114770 0.124 227.0 1.7 645.5 259.7\n', '000000114770 0.103 4.0 -2.0 638.1 387.2\n', '000000114770 0.102 300.2 3.0 637.8 264.4\n', '000000114770 0.095 126.0 9.6 636.7 270.3\n', '000000114770 0.089 188.0 11.7 630.3 204.9\n', '000000114770 0.085 455.8 2.5 636.6 181.5\n', '000000493286 0.680 -0.9 40.4 637.0 207.1\n', '000000493286 0.522 597.7 166.0 640.4 193.0\n', '000000493286 0.429 596.5 162.7 640.8 189.9\n', '000000493286 0.312 2.3 149.8 130.8 196.9\n', '000000493286 0.187 602.4 149.6 640.7 170.4\n', '000000493286 0.184 599.7 161.8 631.8 173.6\n', '000000493286 0.143 565.7 158.0 638.5 179.5\n', '000000493286 0.128 0.5 35.3 639.4 347.1\n', '000000178982 0.098 510.3 2.6 640.2 90.9\n', '000000178982 0.073 506.6 10.1 618.6 76.8\n', '000000382122 0.210 212.0 79.5 221.0 94.0\n', '000000382122 0.109 212.4 78.6 222.7 94.0\n', '000000484760 0.115 182.9 23.2 202.5 42.4\n', '000000469192 0.081 632.7 204.0 640.4 216.0\n', '000000059920 0.062 0.9 14.6 426.2 502.9\n', '000000058029 0.087 413.1 28.8 481.7 62.4\n', '000000058029 0.064 387.1 44.9 415.9 70.5\n', '000000268831 0.092 152.3 146.4 182.3 181.1\n', '000000268831 0.083 146.9 145.5 234.0 181.8\n', '000000175387 0.351 320.1 290.0 500.3 373.1\n', '000000175387 0.069 321.4 262.8 495.8 370.3\n', '000000175387 0.065 314.8 174.8 498.7 372.6\n', '000000534664 0.182 1.0 0.9 640.0 403.7\n', '000000534664 0.084 -2.6 5.5 636.4 276.8\n', '000000278353 0.059 83.2 36.1 534.4 609.5\n', '000000388258 0.652 498.9 91.6 563.8 122.0\n', '000000463037 0.689 7.7 97.1 638.0 412.7\n', '000000463037 0.213 611.9 241.6 639.4 257.4\n', '000000134886 0.679 140.9 184.0 369.7 252.5\n', '000000404479 0.717 184.5 230.3 623.5 367.8\n', '000000513580 0.684 132.1 112.8 508.5 323.1\n', '000000323709 0.739 61.0 127.0 469.2 257.3\n', '000000323709 0.060 92.9 210.7 327.9 253.6\n', '000000096549 0.731 71.7 11.8 633.2 170.8\n', '000000005477 0.720 11.9 108.6 623.1 283.5\n', '000000005477 0.563 43.9 248.3 147.8 275.1\n', '000000005477 0.177 46.1 250.7 99.3 272.1\n', '000000005477 0.154 589.9 254.3 642.6 269.4\n', '000000005477 0.152 126.1 249.6 207.5 273.8\n', '000000005477 0.150 42.0 252.3 95.8 273.1\n', '000000005477 0.133 590.1 259.2 642.9 271.1\n', '000000005477 0.130 591.7 258.2 641.1 268.6\n', '000000005477 0.120 125.2 252.5 205.5 273.5\n', '000000005477 0.090 124.2 249.0 165.3 271.4\n', '000000005477 0.088 622.6 229.3 643.0 262.4\n', '000000005477 0.085 34.3 265.4 57.8 271.7\n', '000000005477 0.071 46.3 248.4 78.8 267.3\n', '000000005477 0.070 129.7 249.7 163.1 269.2\n', '000000469174 0.694 269.9 73.4 480.0 140.4\n', '000000442480 0.252 197.3 127.3 440.6 282.2\n', '000000442480 0.202 306.0 129.7 476.8 277.8\n', '000000442480 0.128 204.0 124.8 453.7 278.0\n', '000000442480 0.127 306.0 150.4 416.5 272.6\n', '000000442480 0.113 242.4 164.8 430.1 292.0\n', '000000281693 0.722 13.1 62.6 628.7 324.4\n', '000000281693 0.416 558.4 252.3 640.0 335.9\n', '000000281693 0.291 577.5 133.8 638.9 206.6\n', '000000281693 0.183 2.6 231.3 154.3 317.3\n', '000000281693 0.171 9.1 133.8 68.8 222.1\n', '000000281693 0.158 11.5 265.1 154.3 322.8\n', '000000281693 0.156 495.0 256.0 630.2 320.6\n', '000000281693 0.134 540.1 135.1 637.5 335.7\n', '000000281693 0.108 5.8 219.3 91.8 256.4\n', '000000161044 0.668 307.8 158.8 331.4 185.0\n', '000000161044 0.640 360.2 182.2 384.7 206.8\n', '000000161044 0.136 357.4 185.0 381.4 207.0\n', '000000161044 0.117 307.1 159.6 332.4 184.8\n', '000000109900 0.320 78.8 157.2 327.2 245.5\n', '000000109900 0.230 83.0 167.0 373.7 244.0\n', '000000109900 0.174 226.0 136.6 386.5 229.9\n', '000000109900 0.167 55.9 134.3 389.1 218.1\n', '000000109900 0.165 57.9 151.6 361.2 246.4\n', '000000109900 0.112 35.9 144.3 327.8 214.8\n', '000000567432 0.687 169.1 231.9 378.2 412.7\n', '000000517523 0.693 48.3 30.9 610.1 216.6\n', '000000459467 0.594 116.6 182.4 475.4 255.2\n', '000000525322 0.676 231.7 200.5 443.4 351.5\n', '000000525322 0.481 118.5 143.3 355.6 256.6\n', '000000525322 0.255 122.5 144.5 439.8 268.9\n', '000000525322 0.159 106.1 143.7 454.5 274.8\n', '000000525322 0.147 139.3 194.0 190.5 219.6\n', '000000525322 0.131 123.0 195.3 210.0 248.5\n', '000000525322 0.118 121.9 139.9 459.2 347.0\n', '000000525322 0.114 154.3 146.7 427.3 264.4\n', '000000525322 0.102 131.9 193.9 175.8 221.6\n', '000000525322 0.086 199.0 164.0 444.4 349.8\n', '000000525322 0.081 298.5 206.3 368.0 249.2\n', '000000485237 0.581 216.3 42.2 528.2 139.3\n', '000000485237 0.490 43.4 9.3 318.8 114.2\n', '000000131386 0.724 151.3 195.5 529.5 381.4\n', '000000131386 0.459 374.8 188.5 568.4 356.0\n', '000000131386 0.459 443.0 187.8 642.1 344.5\n', '000000131386 0.385 510.8 188.1 641.8 342.0\n', '000000131386 0.225 377.1 187.2 531.5 286.9\n', '000000131386 0.219 312.7 199.1 475.7 282.1\n', '000000131386 0.202 516.3 188.3 640.8 287.8\n', '000000131386 0.135 435.8 191.7 569.0 351.1\n', '000000131386 0.129 314.3 211.1 467.8 284.8\n', '000000131386 0.125 378.2 192.3 463.7 267.1\n', '000000131386 0.123 445.4 193.7 599.8 288.6\n', '000000131386 0.121 451.9 185.8 527.6 280.2\n', '000000131386 0.121 585.6 204.6 640.7 281.9\n', '000000131386 0.113 592.2 291.9 643.2 335.7\n', '000000131386 0.112 379.8 190.3 526.7 271.7\n', '000000131386 0.109 381.5 193.3 471.7 258.7\n', '000000131386 0.103 296.6 192.0 545.9 362.4\n', '000000131386 0.100 447.0 234.1 567.8 352.2\n', '000000131386 0.095 584.7 206.8 642.2 333.1\n', '000000131386 0.090 414.7 204.0 568.2 353.6\n', '000000131386 0.087 591.6 204.2 641.3 286.8\n', '000000131386 0.085 203.9 194.0 489.5 289.1\n', '000000131386 0.082 591.3 325.1 641.5 338.8\n', '000000131386 0.078 314.0 206.4 455.1 273.9\n', '000000500049 0.675 6.6 79.9 488.0 258.2\n', '000000500049 0.152 461.1 144.0 497.8 205.5\n', '000000500049 0.138 461.0 148.4 496.8 180.8\n', '000000022396 0.661 326.3 43.5 603.7 156.2\n', '000000101787 0.703 178.6 282.1 222.1 315.0\n', '000000488270 0.713 349.0 225.5 486.6 273.0\n', '000000488270 0.242 469.4 239.6 499.7 249.9\n', '000000488270 0.091 446.5 230.7 499.5 249.2\n', '000000425221 0.511 199.3 410.1 410.1 504.8\n', '000000425221 0.466 152.9 296.8 213.6 314.3\n', '000000425221 0.139 290.8 450.9 365.5 465.8\n', '000000425221 0.110 286.4 448.9 358.3 464.7\n', '000000425221 0.096 202.4 404.5 374.1 467.8\n', '000000425221 0.088 222.5 433.6 436.8 492.1\n', '000000425221 0.087 206.7 411.1 257.7 470.1\n', '000000425221 0.080 206.0 414.1 439.0 505.8\n', '000000425221 0.069 35.4 438.2 453.8 634.7\n', '000000425221 0.068 257.4 497.4 460.1 624.6\n', '000000199977 0.694 83.5 150.4 230.1 318.8\n', '000000449996 0.684 142.5 247.4 263.5 294.9\n', '000000449996 0.681 112.4 291.5 247.6 337.3\n', '000000449996 0.645 441.8 114.3 501.5 128.4\n', '000000449996 0.454 549.0 250.1 643.3 292.7\n', '000000449996 0.335 393.3 274.9 463.1 294.6\n', '000000449996 0.273 499.2 267.8 578.4 295.2\n', '000000449996 0.265 556.8 256.7 578.7 288.7\n', '000000449996 0.147 444.7 274.5 459.6 292.4\n', '000000449996 0.143 555.9 260.5 646.2 292.2\n', '000000137950 0.699 10.3 30.3 526.8 173.2\n', '000000135673 0.692 217.5 162.0 347.8 210.2\n', '000000135673 0.689 312.4 199.7 421.8 249.6\n', '000000490413 0.659 15.2 43.7 636.3 210.1\n', '000000490413 0.275 14.2 133.8 379.5 217.8\n', '000000490413 0.127 20.1 126.7 422.8 216.0\n', '000000490413 0.089 274.9 45.1 637.3 201.4\n', '000000490413 0.075 369.4 44.5 636.8 185.9\n', '000000217060 0.556 245.3 301.8 330.4 376.3\n', '000000208208 0.621 274.1 -0.6 635.7 241.1\n', '000000208208 0.095 281.4 6.9 635.8 468.1\n', '000000208208 0.065 273.7 5.2 640.0 450.3\n', '000000545407 0.716 178.7 172.8 475.8 297.6\n', '000000293324 0.701 341.8 223.5 577.0 328.3\n', '000000293324 0.501 79.8 237.4 391.9 327.3\n', '000000293324 0.271 28.7 278.8 62.6 302.8\n', '000000293324 0.266 85.0 233.6 277.4 324.5\n', '000000293324 0.254 187.0 260.2 417.7 324.0\n', '000000293324 0.233 24.3 277.0 153.1 327.5\n', '000000293324 0.210 235.6 272.3 247.4 281.9\n', '000000293324 0.207 83.5 234.8 216.1 312.7\n', '000000293324 0.197 89.7 236.6 193.9 306.1\n', '000000293324 0.181 65.9 240.5 428.8 327.9\n', '000000293324 0.174 131.8 250.4 420.4 319.3\n', '000000383621 0.716 37.6 207.8 452.1 299.5\n', '000000336309 0.700 121.7 187.4 288.5 241.3\n', '000000497568 0.718 5.3 74.3 636.5 400.5\n', '000000497568 0.089 0.9 181.8 11.0 199.7\n', '000000497568 0.082 0.9 183.3 10.6 199.7\n', '000000272136 0.523 83.1 50.3 643.8 421.9\n', '000000272136 0.156 455.1 53.7 639.3 126.1\n', '000000272136 0.137 98.5 49.7 633.6 210.0\n', '000000272136 0.137 284.5 48.3 636.7 195.7\n', '000000272136 0.108 459.9 55.0 637.3 129.7\n', '000000272136 0.096 291.8 49.8 641.0 197.7\n', '000000084752 0.704 27.2 106.8 598.3 322.8\n', '000000196185 0.714 105.2 155.8 532.0 278.8\n', '000000196185 0.295 505.2 330.7 555.4 365.9\n', '000000196185 0.179 522.5 332.0 578.2 360.0\n', '000000196185 0.145 501.3 322.3 555.7 349.6\n', '000000196185 0.139 506.0 327.9 518.1 343.0\n', '000000196185 0.132 432.9 345.9 448.0 369.0\n', '000000196185 0.123 414.3 320.7 562.4 381.9\n', '000000196185 0.107 555.4 327.4 573.4 338.7\n', '000000196185 0.099 587.8 337.5 616.2 349.1\n', '000000196185 0.099 451.1 347.8 523.9 391.9\n', '000000196185 0.074 418.6 356.8 524.5 396.8\n', '000000196185 0.073 617.2 331.5 637.2 342.6\n', '000000575205 0.300 122.0 212.9 470.7 280.0\n', '000000456865 0.733 89.7 160.0 587.2 313.5\n', '000000456865 0.625 22.3 255.9 65.0 270.8\n', '000000456865 0.300 245.6 183.8 261.4 188.5\n', '000000456865 0.133 62.1 265.3 94.9 274.5\n', '000000456865 0.100 21.0 258.1 33.1 267.5\n', '000000456865 0.096 244.7 181.6 259.8 186.1\n', '000000456865 0.095 63.2 264.8 83.0 273.6\n', '000000456865 0.086 62.5 266.7 88.9 276.5\n', '000000456865 0.079 15.4 257.4 32.5 268.0\n', '000000456865 0.072 22.8 253.5 64.1 270.3\n', '000000424776 0.727 23.8 84.3 606.3 328.7\n', '000000404128 0.724 32.1 145.8 523.0 335.8\n', '000000381639 0.284 65.1 16.0 464.3 129.1\n', '000000381639 0.205 33.1 38.5 208.8 143.0\n', '000000381639 0.182 411.0 93.2 483.8 136.5\n', '000000381639 0.141 34.5 32.5 496.6 124.3\n', '000000381639 0.136 15.1 122.3 108.7 166.1\n', '000000381639 0.133 11.6 31.5 568.8 130.9\n', '000000187745 0.689 207.4 166.0 323.5 234.9\n', '000000071711 0.345 2.1 175.9 205.0 326.2\n', '000000071711 0.272 12.2 177.7 383.8 335.6\n', '000000071711 0.224 8.8 176.9 252.2 341.3\n', '000000071711 0.210 12.8 180.1 288.8 339.7\n', '000000071711 0.210 371.5 58.8 637.3 319.0\n', '000000071711 0.206 246.3 25.2 622.7 320.0\n', '000000071711 0.193 297.5 53.0 638.8 326.1\n', '000000071711 0.188 3.4 14.3 642.3 335.0\n', '000000071711 0.177 11.8 -5.8 440.7 336.2\n', '000000071711 0.172 1.0 2.7 640.0 327.1\n', '000000189828 0.656 362.9 99.5 569.7 205.2\n', '000000099054 0.495 2.7 214.4 427.1 554.0\n', '000000099054 0.317 31.5 71.3 427.8 558.2\n', '000000099054 0.260 309.8 212.8 363.7 233.5\n', '000000099054 0.229 1.3 86.7 426.2 558.5\n', '000000099054 0.196 53.6 111.4 430.2 318.2\n', '000000099054 0.168 4.4 154.4 428.7 548.5\n', '000000099054 0.120 386.9 221.6 424.5 235.2\n', '000000099054 0.117 333.7 215.8 366.7 233.3\n', '000000099054 0.113 67.8 116.5 422.5 416.4\n', '000000099054 0.112 387.3 223.2 412.3 233.4\n', '000000110721 0.648 18.0 0.9 638.2 357.2\n', '000000110721 0.157 119.7 1.6 154.4 19.2\n', '000000110721 0.109 141.5 2.3 158.9 19.6\n', '000000110721 0.100 141.5 1.1 170.0 13.5\n', '000000110721 0.090 76.2 1.6 121.7 21.6\n', '000000110721 0.089 108.4 1.2 135.7 16.6\n', '000000338325 0.698 117.9 137.8 453.4 304.1\n', '000000208901 0.695 233.6 146.5 424.9 284.7\n', '000000110359 0.676 219.7 146.9 422.2 384.7\n', '000000001761 0.667 278.4 138.8 334.0 186.1\n', '000000001761 0.638 150.1 7.4 224.7 92.3\n', '000000090631 0.720 178.9 119.0 451.4 229.3\n', '000000090631 0.097 177.3 202.3 247.6 227.7\n', '000000553094 0.655 173.2 47.2 451.9 405.3\n', '000000502599 0.668 126.8 292.7 386.7 402.2\n', '000000502599 0.571 393.1 323.6 591.6 410.8\n', '000000502599 0.569 18.5 124.2 464.2 228.9\n', '000000502599 0.551 2.0 210.0 187.7 308.4\n', '000000502599 0.149 4.6 3.8 288.5 152.6\n', '000000502599 0.113 0.7 319.9 27.4 347.4\n', '000000502599 0.105 428.9 -3.7 640.6 418.3\n', '000000502599 0.104 7.0 7.8 318.5 159.7\n', '000000502599 0.095 226.5 121.5 318.3 144.6\n', '000000502599 0.087 6.9 19.9 416.2 157.1\n', '000000396903 0.689 25.9 274.8 405.1 402.3\n', '000000229747 0.693 173.4 191.6 422.8 345.1\n', '000000139871 0.725 18.6 95.1 624.7 310.9\n', '000000502347 0.633 269.2 85.5 297.0 98.5\n', '000000452122 0.714 76.9 126.5 553.2 275.5\n', '000000190676 0.622 8.1 67.7 215.9 145.7\n', '000000477441 0.726 95.2 154.7 410.9 292.6\n', '000000477441 0.134 76.6 135.1 616.6 296.8\n', '000000477441 0.098 307.6 183.8 346.6 228.3\n', '000000205401 0.183 157.0 138.4 537.6 271.3\n', '000000205401 0.085 110.0 189.9 168.6 213.8\n', '000000408112 0.729 0.0 104.6 412.5 323.8\n', '000000408112 0.680 415.7 168.6 626.0 233.2\n', '000000408112 0.207 623.4 163.5 641.1 207.2\n', '000000408112 0.186 623.9 161.2 641.2 201.2\n', '000000408112 0.163 407.4 174.6 575.5 232.9\n', '000000408112 0.132 472.6 165.4 623.1 214.5\n', '000000408112 0.123 415.7 186.8 500.7 227.6\n', '000000408112 0.098 264.8 204.5 315.9 215.1\n', '000000408112 0.096 262.0 204.1 330.4 213.6\n', '000000479912 0.672 -0.2 222.3 339.0 446.9\n', '000000479912 0.460 113.9 261.0 146.5 278.9\n', '000000479912 0.176 129.0 261.4 194.2 280.7\n', '000000479912 0.148 150.4 264.0 190.3 277.9\n', '000000479912 0.147 54.4 265.0 89.6 274.0\n', '000000479912 0.140 53.7 265.7 88.7 274.5\n', '000000479912 0.129 0.5 321.3 17.3 333.9\n', '000000479912 0.128 95.3 265.2 117.5 276.5\n', '000000479912 0.117 154.2 265.4 194.5 278.6\n', '000000479912 0.083 190.1 269.9 210.6 283.7\n', '000000479912 0.083 1.9 263.7 16.5 272.5\n', '000000052017 0.729 58.6 80.7 478.1 289.2\n', '000000163746 0.511 175.9 -0.2 230.2 61.3\n', '000000163746 0.455 520.8 -0.0 575.1 61.1\n', '000000033114 0.713 170.6 170.5 504.4 276.7\n', '000000013348 0.727 7.5 135.0 572.0 283.0\n', '000000013348 0.450 1.6 224.9 52.9 244.1\n', '000000013348 0.222 1.1 220.5 47.0 242.1\n', '000000543528 0.707 186.0 78.9 519.4 245.6\n', '000000167540 0.722 169.1 79.8 489.8 326.9\n', '000000384350 0.652 3.6 47.8 639.1 249.2\n', '000000384350 0.190 194.8 248.7 625.5 338.6\n', '000000384350 0.105 204.2 249.6 632.7 349.4\n', '000000384350 0.101 194.7 251.5 575.6 334.7\n', '000000384350 0.071 193.1 254.0 598.4 344.6\n', '000000044652 0.695 75.3 165.1 270.9 248.7\n', '000000044652 0.084 146.8 172.5 268.0 248.5\n', '000000044652 0.067 77.2 179.4 135.4 231.1\n', '000000410221 0.655 270.1 137.6 377.1 254.9\n', '000000478862 0.704 9.3 55.5 642.0 341.9\n', '000000579158 0.725 2.0 3.0 589.9 424.6\n', '000000579158 0.285 519.7 338.7 646.5 384.6\n', '000000579158 0.166 539.6 345.1 642.2 396.2\n', '000000579158 0.161 526.2 241.6 639.3 390.1\n', '000000579158 0.135 1.0 228.9 193.0 333.4\n', '000000579158 0.127 519.2 329.0 631.0 382.5\n', '000000579158 0.121 530.0 119.4 637.2 394.2\n', '000000579158 0.115 518.3 313.3 639.9 381.4\n', '000000579158 0.110 3.8 231.1 192.5 331.3\n', '000000579158 0.105 536.3 338.0 646.7 483.2\n', '000000214205 0.711 252.5 310.8 400.7 360.6\n', '000000379453 0.703 123.8 123.6 350.4 309.6\n', '000000144114 0.705 34.9 178.1 581.3 337.4\n', '000000144114 0.100 542.0 189.6 575.9 203.9\n', '000000520324 0.673 142.1 216.0 635.7 350.5\n', '000000520324 0.280 145.6 222.1 350.3 276.4\n', '000000520324 0.091 584.9 256.8 640.8 288.9\n', '000000520324 0.081 270.5 245.1 638.1 350.8\n', '000000520324 0.078 161.1 219.2 579.6 271.1\n', '000000520324 0.077 142.3 266.7 239.3 329.8\n', '000000098520 0.722 88.9 126.6 616.2 285.5\n', '000000504000 0.574 17.9 45.4 652.6 217.9\n', '000000504000 0.508 -1.4 48.6 94.6 139.4\n', '000000210502 0.090 0.6 26.7 639.6 451.1\n', '000000348881 0.362 1.4 42.4 38.2 72.0\n', '000000348881 0.335 13.0 48.8 160.2 231.0\n', '000000348881 0.276 6.8 24.8 83.3 57.5\n', '000000348881 0.229 0.4 25.4 44.9 58.3\n', '000000348881 0.186 0.5 25.7 80.1 71.9\n', '000000348881 0.173 11.4 1.2 156.6 228.5\n', '000000348881 0.165 120.1 46.9 283.6 208.1\n', '000000348881 0.144 13.1 23.0 279.3 222.4\n', '000000348881 0.131 15.2 38.4 262.4 240.2\n', '000000348881 0.121 -1.6 35.2 290.3 223.9\n', '000000348881 0.105 578.4 195.2 641.6 242.8\n', '000000571857 0.532 260.1 179.7 448.0 231.1\n', '000000571857 0.425 22.5 160.2 485.3 239.1\n', '000000571857 0.333 441.4 192.7 495.8 207.0\n', '000000571857 0.330 2.6 186.3 87.5 227.7\n', '000000571857 0.296 2.8 152.9 501.7 261.3\n', '000000571857 0.161 344.6 189.1 430.1 211.5\n', '000000571857 0.155 339.4 180.7 497.8 210.3\n', '000000571857 0.154 6.1 154.4 249.8 237.8\n', '000000571857 0.131 9.2 161.5 494.7 241.8\n', '000000571857 0.114 329.3 190.2 408.1 207.1\n', '000000571857 0.110 19.6 150.2 370.2 251.7\n', '000000571857 0.108 323.9 192.6 438.3 227.2\n', '000000571857 0.102 8.1 160.5 288.5 242.9\n', '000000571857 0.098 15.9 158.4 478.6 279.0\n', '000000571857 0.097 0.2 158.8 517.6 272.2\n', '000000571857 0.093 0.9 191.9 22.4 214.6\n', '000000571857 0.091 357.2 187.6 431.3 202.2\n', '000000571857 0.088 -1.8 154.8 480.0 262.6\n', '000000571857 0.084 -0.3 158.1 89.2 227.6\n', '000000571857 0.082 459.0 192.6 501.2 206.0\n', '000000571857 0.080 14.4 152.7 415.8 242.1\n', '000000571857 0.078 0.9 188.3 22.7 207.7\n', '000000571857 0.078 353.3 168.1 506.0 207.4\n', '000000183500 0.742 91.1 80.2 538.2 343.3\n', '000000485802 0.656 207.1 193.7 222.4 209.6\n', '000000485802 0.106 186.1 204.5 213.2 312.2\n', '000000495054 0.718 66.1 130.2 607.4 278.5\n', '000000269682 0.056 333.6 218.4 353.7 242.8\n', '000000271457 0.156 0.7 1.9 499.6 331.4\n', '000000271457 0.103 0.3 129.3 499.3 326.5\n', '000000271457 0.095 0.4 84.0 498.0 331.8\n', '000000271457 0.084 -0.8 111.7 497.9 328.9\n', '000000322829 0.116 82.6 50.2 528.2 304.6\n', '000000322829 0.066 85.1 52.2 535.7 369.7\n', '000000290619 0.074 447.0 274.3 488.1 312.2\n', '000000300659 0.666 203.6 193.5 234.7 240.1\n', '000000300659 0.647 255.5 178.8 285.0 227.2\n', '000000300659 0.640 312.1 221.9 340.5 270.2\n', '000000300659 0.621 308.8 164.4 338.0 212.4\n', '000000300659 0.617 306.5 280.9 335.8 330.8\n', '000000300659 0.609 229.4 61.0 255.7 110.0\n', '000000300659 0.541 270.2 114.2 296.7 163.5\n', '000000300659 0.090 303.7 288.8 331.7 319.7\n', '000000300659 0.086 86.3 51.4 233.0 112.1\n', '000000300659 0.078 302.6 286.5 329.8 328.0\n', '000000392481 0.703 339.2 204.1 466.7 299.7\n', '000000135872 0.119 1.1 293.3 151.0 366.4\n', '000000157847 0.215 374.4 95.1 402.5 114.1\n', '000000200667 0.132 0.4 2.1 639.4 422.4\n', '000000200667 0.088 349.3 4.3 618.7 326.3\n', '000000039484 0.267 606.0 13.9 633.5 23.2\n', '000000272049 0.577 1.1 34.6 205.6 108.6\n', '000000272049 0.285 18.1 105.3 246.2 240.4\n', '000000272049 0.137 19.8 131.1 156.3 240.1\n', '000000272049 0.090 16.6 35.2 230.2 110.0\n', '000000304365 0.081 191.8 218.8 209.7 226.7\n', '000000042178 0.089 34.3 205.3 638.8 474.0\n', '000000249643 0.113 580.7 178.4 640.3 274.5\n', '000000249643 0.091 503.1 185.7 586.9 267.4\n', '000000225946 0.080 185.8 157.3 507.1 274.4\n', '000000478286 0.138 201.8 81.6 251.0 102.9\n', '000000368038 0.066 82.7 18.7 633.9 290.5\n', '000000057244 0.171 0.3 39.5 639.2 424.7\n', '000000184400 0.084 390.3 92.8 434.6 112.5\n', '000000475365 0.084 3.5 1.3 461.5 423.3\n', '000000377882 0.109 474.6 125.4 643.0 277.7\n', '000000234660 0.243 237.2 104.5 643.5 467.9\n', '000000234660 0.204 33.1 94.9 626.7 316.7\n', '000000273617 0.081 474.0 52.9 525.4 105.5\n', '000000046872 0.094 -1.0 83.7 504.0 387.4\n', '000000205289 0.084 147.6 122.4 488.2 285.5\n', '000000333956 0.094 0.1 314.5 50.3 368.2\n', '000000301061 0.268 128.2 -6.1 479.0 631.9\n', '000000174004 0.082 17.5 105.3 616.3 367.0\n', '000000106281 0.129 155.3 42.5 232.0 64.2\n', '000000106281 0.089 141.5 40.3 243.8 73.6\n', '000000041635 0.206 371.8 86.2 425.2 101.1\n', '000000041635 0.167 372.3 89.7 381.4 100.2\n', '000000041635 0.146 411.4 87.6 425.5 99.2\n', '000000041635 0.118 462.8 86.8 470.3 96.6\n', '000000390826 0.076 190.2 138.8 331.7 376.8\n', '000000061418 0.208 158.4 138.5 248.5 175.7\n', '000000503823 0.082 418.2 125.1 464.9 154.3\n', '000000365098 0.057 40.4 100.7 635.7 403.0\n', '000000561335 0.099 207.4 76.5 385.6 322.3\n', '000000369037 0.100 -1.0 5.2 422.8 634.3\n', '000000519208 0.069 1.4 0.8 640.4 405.5\n', '000000308328 0.100 4.2 43.6 637.5 471.6\n', '000000308328 0.094 70.3 56.0 602.1 470.3\n', '000000341094 0.259 3.2 67.8 502.1 366.5\n', '000000341094 0.086 1.0 30.7 499.9 368.6\n', '000000307658 0.059 8.2 52.3 537.8 636.5\n', '000000079144 0.099 137.2 136.8 271.0 264.0\n', '000000079144 0.071 251.1 175.2 343.8 367.1\n', '000000085478 0.068 27.7 81.2 218.8 326.6\n', '000000308753 0.175 250.7 226.1 412.9 279.7\n', '000000308753 0.069 375.5 443.0 645.4 480.5\n', '000000194506 0.067 71.2 114.9 307.0 274.2\n', '000000194216 0.089 489.5 152.5 636.0 257.9\n', '000000359855 0.087 139.0 13.6 563.0 330.1\n', '000000226592 0.100 100.6 110.6 645.1 451.6\n', '000000172935 0.299 19.0 100.3 270.7 216.0\n', '000000172935 0.071 77.2 142.9 393.9 223.9\n', '000000125211 0.100 75.8 95.0 636.8 479.0\n', '000000215245 0.086 162.5 199.2 207.7 231.6\n', '000000327780 0.072 0.9 1.3 639.9 479.9\n', '000000558213 0.087 342.7 189.6 637.8 302.7\n', '000000435081 0.100 1.9 2.3 499.6 499.4\n', '000000161799 0.526 472.8 73.4 507.8 88.4\n', '000000185292 0.067 1.9 140.4 421.7 193.1\n', '000000331604 0.067 336.8 106.1 369.6 150.3\n', '000000130613 0.076 215.1 74.6 489.6 353.6\n', '000000468245 0.094 326.5 1.0 519.1 39.1\n', '000000160666 0.382 3.8 78.5 639.7 263.9\n', '000000160666 0.174 -5.3 83.0 633.6 388.4\n', '000000160666 0.110 0.8 75.6 639.7 439.7\n', '000000160666 0.078 119.9 86.4 642.6 262.3\n', '000000160666 0.078 28.6 90.5 637.0 364.7\n', '000000160666 0.068 -7.7 78.9 631.2 426.9\n', '000000128675 0.089 320.9 13.5 357.3 85.9\n', '000000322895 0.101 398.1 55.2 406.0 59.1\n', '000000322895 0.082 382.5 61.4 388.0 65.7\n', '000000400922 0.648 379.4 125.2 406.8 138.4\n', '000000140929 0.134 195.4 7.2 630.1 277.4\n', '000000140929 0.122 238.7 7.4 633.9 277.0\n', '000000140929 0.114 171.8 12.3 636.0 341.5\n', '000000140929 0.072 193.1 15.8 643.9 343.2\n', '000000140929 0.070 1.1 2.5 639.7 395.8\n', '000000236599 0.229 463.3 67.5 501.0 92.6\n', '000000236599 0.212 424.1 49.4 476.5 81.8\n', '000000007784 0.059 1.3 83.2 335.6 371.9\n', '000000186345 0.437 278.3 56.4 290.9 70.6\n', '000000186345 0.392 319.6 41.6 329.6 58.3\n', '000000186345 0.225 277.1 67.8 290.3 77.9\n', '000000186345 0.137 278.0 56.4 289.8 69.8\n', '000000186345 0.130 261.2 84.1 275.2 101.2\n', '000000207306 0.086 162.4 148.4 212.0 175.1\n', '000000137294 0.136 1.1 0.6 640.0 426.2\n', '000000137294 0.105 2.3 3.1 405.1 428.6\n', '000000322844 0.082 169.4 305.4 479.2 549.3\n', '000000546826 0.159 1.7 1.1 640.7 480.0\n', '000000263425 0.136 35.3 146.4 610.0 423.2\n', '000000263425 0.075 5.2 5.4 636.2 427.6\n', '000000263425 0.075 38.5 105.6 605.4 429.2\n', '000000263425 0.074 61.1 134.0 618.9 425.6\n']

In [155]:
l2=['000000575970 0.093 241.1 87.0 280.9 95.9\n', '000000575970 0.085 261.2 105.2 277.0 120.5\n', '000000575970 0.080 242.6 107.0 257.5 119.2\n', '000000575970 0.078 481.0 89.3 516.1 98.0\n', '000000575970 0.077 276.7 79.8 296.6 89.3\n', '000000519569 0.088 146.7 149.6 158.3 160.3\n', '000000239627 0.098 315.0 149.2 519.5 383.5\n', '000000239627 0.079 311.4 150.4 542.6 403.7\n', '000000135670 0.126 1.1 12.9 375.1 474.7\n', '000000114770 0.199 294.5 -0.9 629.1 200.0\n', '000000114770 0.195 297.4 1.9 632.9 241.6\n', '000000114770 0.124 227.0 1.7 645.5 259.7\n', '000000114770 0.103 4.0 -2.0 638.1 387.2\n', '000000114770 0.102 300.2 3.0 637.8 264.4\n', '000000114770 0.095 126.0 9.6 636.7 270.3\n', '000000114770 0.089 188.0 11.7 630.3 204.9\n', '000000114770 0.085 455.8 2.5 636.6 181.5\n', '000000493286 0.680 -0.9 40.4 637.0 207.1\n', '000000493286 0.522 597.7 166.0 640.4 193.0\n', '000000493286 0.429 596.5 162.7 640.8 189.9\n', '000000493286 0.312 2.3 149.8 130.8 196.9\n', '000000493286 0.187 602.4 149.6 640.7 170.4\n', '000000493286 0.184 599.7 161.8 631.8 173.6\n', '000000493286 0.143 565.7 158.0 638.5 179.5\n', '000000493286 0.128 0.5 35.3 639.4 347.1\n', '000000178982 0.098 510.3 2.6 640.2 90.9\n', '000000178982 0.073 506.6 10.1 618.6 76.8\n', '000000382122 0.210 212.0 79.5 221.0 94.0\n', '000000382122 0.109 212.4 78.6 222.7 94.0\n', '000000484760 0.115 182.9 23.2 202.5 42.4\n', '000000469192 0.081 632.7 204.0 640.4 216.0\n', '000000059920 0.062 0.9 14.6 426.2 502.9\n', '000000058029 0.087 413.1 28.8 481.7 62.4\n', '000000058029 0.064 387.1 44.9 415.9 70.5\n', '000000268831 0.092 152.3 146.4 182.3 181.1\n', '000000268831 0.083 146.9 145.5 234.0 181.8\n', '000000175387 0.351 320.1 290.0 500.3 373.1\n', '000000175387 0.069 321.4 262.8 495.8 370.3\n', '000000175387 0.065 314.8 174.8 498.7 372.6\n', '000000534664 0.182 1.0 0.9 640.0 403.7\n', '000000534664 0.084 -2.6 5.5 636.4 276.8\n', '000000278353 0.059 83.2 36.1 534.4 609.5\n', '000000388258 0.652 498.9 91.6 563.8 122.0\n', '000000463037 0.689 7.7 97.1 638.0 412.7\n', '000000463037 0.213 611.9 241.6 639.4 257.4\n', '000000134886 0.679 140.9 184.0 369.7 252.5\n', '000000404479 0.717 184.5 230.3 623.5 367.8\n', '000000513580 0.684 132.1 112.8 508.5 323.1\n', '000000323709 0.739 61.0 127.0 469.2 257.3\n', '000000323709 0.060 92.9 210.7 327.9 253.6\n', '000000096549 0.731 71.7 11.8 633.2 170.8\n', '000000005477 0.720 11.9 108.6 623.1 283.5\n', '000000005477 0.563 43.9 248.3 147.8 275.1\n', '000000005477 0.177 46.1 250.7 99.3 272.1\n', '000000005477 0.154 589.9 254.3 642.6 269.4\n', '000000005477 0.152 126.1 249.6 207.5 273.8\n', '000000005477 0.150 42.0 252.3 95.8 273.1\n', '000000005477 0.133 590.1 259.2 642.9 271.1\n', '000000005477 0.130 591.7 258.2 641.1 268.6\n', '000000005477 0.120 125.2 252.5 205.5 273.5\n', '000000005477 0.090 124.2 249.0 165.3 271.4\n', '000000005477 0.088 622.6 229.3 643.0 262.4\n', '000000005477 0.085 34.3 265.4 57.8 271.7\n', '000000005477 0.071 46.3 248.4 78.8 267.3\n', '000000005477 0.070 129.7 249.7 163.1 269.2\n', '000000469174 0.694 269.9 73.4 480.0 140.4\n', '000000442480 0.252 197.3 127.3 440.6 282.2\n', '000000442480 0.202 306.0 129.7 476.8 277.8\n', '000000442480 0.128 204.0 124.8 453.7 278.0\n', '000000442480 0.127 306.0 150.4 416.5 272.6\n', '000000442480 0.113 242.4 164.8 430.1 292.0\n', '000000281693 0.722 13.1 62.6 628.7 324.4\n', '000000281693 0.416 558.4 252.3 640.0 335.9\n', '000000281693 0.291 577.5 133.8 638.9 206.6\n', '000000281693 0.183 2.6 231.3 154.3 317.3\n', '000000281693 0.171 9.1 133.8 68.8 222.1\n', '000000281693 0.158 11.5 265.1 154.3 322.8\n', '000000281693 0.156 495.0 256.0 630.2 320.6\n', '000000281693 0.134 540.1 135.1 637.5 335.7\n', '000000281693 0.108 5.8 219.3 91.8 256.4\n', '000000161044 0.668 307.8 158.8 331.4 185.0\n', '000000161044 0.640 360.2 182.2 384.7 206.8\n', '000000161044 0.136 357.4 185.0 381.4 207.0\n', '000000161044 0.117 307.1 159.6 332.4 184.8\n', '000000109900 0.320 78.8 157.2 327.2 245.5\n', '000000109900 0.230 83.0 167.0 373.7 244.0\n', '000000109900 0.174 226.0 136.6 386.5 229.9\n', '000000109900 0.167 55.9 134.3 389.1 218.1\n', '000000109900 0.165 57.9 151.6 361.2 246.4\n', '000000109900 0.112 35.9 144.3 327.8 214.8\n', '000000567432 0.687 169.1 231.9 378.2 412.7\n', '000000517523 0.693 48.3 30.9 610.1 216.6\n', '000000459467 0.594 116.6 182.4 475.4 255.2\n', '000000525322 0.676 231.7 200.5 443.4 351.5\n', '000000525322 0.481 118.5 143.3 355.6 256.6\n', '000000525322 0.255 122.5 144.5 439.8 268.9\n', '000000525322 0.159 106.1 143.7 454.5 274.8\n', '000000525322 0.147 139.3 194.0 190.5 219.6\n', '000000525322 0.131 123.0 195.3 210.0 248.5\n', '000000525322 0.118 121.9 139.9 459.2 347.0\n', '000000525322 0.114 154.3 146.7 427.3 264.4\n', '000000525322 0.102 131.9 193.9 175.8 221.6\n', '000000525322 0.086 199.0 164.0 444.4 349.8\n', '000000525322 0.081 298.5 206.3 368.0 249.2\n', '000000485237 0.581 216.3 42.2 528.2 139.3\n', '000000485237 0.490 43.4 9.3 318.8 114.2\n', '000000131386 0.724 151.3 195.5 529.5 381.4\n', '000000131386 0.459 374.8 188.5 568.4 356.0\n', '000000131386 0.459 443.0 187.8 642.1 344.5\n', '000000131386 0.385 510.8 188.1 641.8 342.0\n', '000000131386 0.225 377.1 187.2 531.5 286.9\n', '000000131386 0.219 312.7 199.1 475.7 282.1\n', '000000131386 0.202 516.3 188.3 640.8 287.8\n', '000000131386 0.135 435.8 191.7 569.0 351.1\n', '000000131386 0.129 314.3 211.1 467.8 284.8\n', '000000131386 0.125 378.2 192.3 463.7 267.1\n', '000000131386 0.123 445.4 193.7 599.8 288.6\n', '000000131386 0.121 451.9 185.8 527.6 280.2\n', '000000131386 0.121 585.6 204.6 640.7 281.9\n', '000000131386 0.113 592.2 291.9 643.2 335.7\n', '000000131386 0.112 379.8 190.3 526.7 271.7\n', '000000131386 0.109 381.5 193.3 471.7 258.7\n', '000000131386 0.103 296.6 192.0 545.9 362.4\n', '000000131386 0.100 447.0 234.1 567.8 352.2\n', '000000131386 0.095 584.7 206.8 642.2 333.1\n', '000000131386 0.090 414.7 204.0 568.2 353.6\n', '000000131386 0.087 591.6 204.2 641.3 286.8\n', '000000131386 0.085 203.9 194.0 489.5 289.1\n', '000000131386 0.082 591.3 325.1 641.5 338.8\n', '000000131386 0.078 314.0 206.4 455.1 273.9\n', '000000500049 0.675 6.6 79.9 488.0 258.2\n', '000000500049 0.152 461.1 144.0 497.8 205.5\n', '000000500049 0.138 461.0 148.4 496.8 180.8\n', '000000022396 0.661 326.3 43.5 603.7 156.2\n', '000000101787 0.703 178.6 282.1 222.1 315.0\n', '000000488270 0.713 349.0 225.5 486.6 273.0\n', '000000488270 0.242 469.4 239.6 499.7 249.9\n', '000000488270 0.091 446.5 230.7 499.5 249.2\n', '000000425221 0.511 199.3 410.1 410.1 504.8\n', '000000425221 0.466 152.9 296.8 213.6 314.3\n', '000000425221 0.139 290.8 450.9 365.5 465.8\n', '000000425221 0.110 286.4 448.9 358.3 464.7\n', '000000425221 0.096 202.4 404.5 374.1 467.8\n', '000000425221 0.088 222.5 433.6 436.8 492.1\n', '000000425221 0.087 206.7 411.1 257.7 470.1\n', '000000425221 0.080 206.0 414.1 439.0 505.8\n', '000000425221 0.069 35.4 438.2 453.8 634.7\n', '000000425221 0.068 257.4 497.4 460.1 624.6\n', '000000199977 0.694 83.5 150.4 230.1 318.8\n', '000000449996 0.684 142.5 247.4 263.5 294.9\n', '000000449996 0.681 112.4 291.5 247.6 337.3\n', '000000449996 0.645 441.8 114.3 501.5 128.4\n', '000000449996 0.454 549.0 250.1 643.3 292.7\n', '000000449996 0.335 393.3 274.9 463.1 294.6\n', '000000449996 0.273 499.2 267.8 578.4 295.2\n', '000000449996 0.265 556.8 256.7 578.7 288.7\n', '000000449996 0.147 444.7 274.5 459.6 292.4\n', '000000449996 0.143 555.9 260.5 646.2 292.2\n', '000000137950 0.699 10.3 30.3 526.8 173.2\n', '000000135673 0.692 217.5 162.0 347.8 210.2\n', '000000135673 0.689 312.4 199.7 421.8 249.6\n', '000000490413 0.659 15.2 43.7 636.3 210.1\n', '000000490413 0.275 14.2 133.8 379.5 217.8\n', '000000490413 0.127 20.1 126.7 422.8 216.0\n', '000000490413 0.089 274.9 45.1 637.3 201.4\n', '000000490413 0.075 369.4 44.5 636.8 185.9\n', '000000217060 0.556 245.3 301.8 330.4 376.3\n', '000000208208 0.621 274.1 -0.6 635.7 241.1\n', '000000208208 0.095 281.4 6.9 635.8 468.1\n', '000000208208 0.065 273.7 5.2 640.0 450.3\n', '000000545407 0.716 178.7 172.8 475.8 297.6\n', '000000293324 0.701 341.8 223.5 577.0 328.3\n', '000000293324 0.501 79.8 237.4 391.9 327.3\n', '000000293324 0.271 28.7 278.8 62.6 302.8\n', '000000293324 0.266 85.0 233.6 277.4 324.5\n', '000000293324 0.254 187.0 260.2 417.7 324.0\n', '000000293324 0.233 24.3 277.0 153.1 327.5\n', '000000293324 0.210 235.6 272.3 247.4 281.9\n', '000000293324 0.207 83.5 234.8 216.1 312.7\n', '000000293324 0.197 89.7 236.6 193.9 306.1\n', '000000293324 0.181 65.9 240.5 428.8 327.9\n', '000000293324 0.174 131.8 250.4 420.4 319.3\n', '000000383621 0.716 37.6 207.8 452.1 299.5\n', '000000336309 0.700 121.7 187.4 288.5 241.3\n', '000000497568 0.718 5.3 74.3 636.5 400.5\n', '000000497568 0.089 0.9 181.8 11.0 199.7\n', '000000497568 0.082 0.9 183.3 10.6 199.7\n', '000000272136 0.523 83.1 50.3 643.8 421.9\n', '000000272136 0.156 455.1 53.7 639.3 126.1\n', '000000272136 0.137 98.5 49.7 633.6 210.0\n', '000000272136 0.137 284.5 48.3 636.7 195.7\n', '000000272136 0.108 459.9 55.0 637.3 129.7\n', '000000272136 0.096 291.8 49.8 641.0 197.7\n', '000000084752 0.704 27.2 106.8 598.3 322.8\n', '000000196185 0.714 105.2 155.8 532.0 278.8\n', '000000196185 0.295 505.2 330.7 555.4 365.9\n', '000000196185 0.179 522.5 332.0 578.2 360.0\n', '000000196185 0.145 501.3 322.3 555.7 349.6\n', '000000196185 0.139 506.0 327.9 518.1 343.0\n', '000000196185 0.132 432.9 345.9 448.0 369.0\n', '000000196185 0.123 414.3 320.7 562.4 381.9\n', '000000196185 0.107 555.4 327.4 573.4 338.7\n', '000000196185 0.099 587.8 337.5 616.2 349.1\n', '000000196185 0.099 451.1 347.8 523.9 391.9\n', '000000196185 0.074 418.6 356.8 524.5 396.8\n', '000000196185 0.073 617.2 331.5 637.2 342.6\n', '000000575205 0.300 122.0 212.9 470.7 280.0\n', '000000456865 0.733 89.7 160.0 587.2 313.5\n', '000000456865 0.625 22.3 255.9 65.0 270.8\n', '000000456865 0.300 245.6 183.8 261.4 188.5\n', '000000456865 0.133 62.1 265.3 94.9 274.5\n', '000000456865 0.100 21.0 258.1 33.1 267.5\n', '000000456865 0.096 244.7 181.6 259.8 186.1\n', '000000456865 0.095 63.2 264.8 83.0 273.6\n', '000000456865 0.086 62.5 266.7 88.9 276.5\n', '000000456865 0.079 15.4 257.4 32.5 268.0\n', '000000456865 0.072 22.8 253.5 64.1 270.3\n', '000000424776 0.727 23.8 84.3 606.3 328.7\n', '000000404128 0.724 32.1 145.8 523.0 335.8\n', '000000381639 0.284 65.1 16.0 464.3 129.1\n', '000000381639 0.205 33.1 38.5 208.8 143.0\n', '000000381639 0.182 411.0 93.2 483.8 136.5\n', '000000381639 0.141 34.5 32.5 496.6 124.3\n', '000000381639 0.136 15.1 122.3 108.7 166.1\n', '000000381639 0.133 11.6 31.5 568.8 130.9\n', '000000187745 0.689 207.4 166.0 323.5 234.9\n', '000000071711 0.345 2.1 175.9 205.0 326.2\n', '000000071711 0.272 12.2 177.7 383.8 335.6\n', '000000071711 0.224 8.8 176.9 252.2 341.3\n', '000000071711 0.210 12.8 180.1 288.8 339.7\n', '000000071711 0.210 371.5 58.8 637.3 319.0\n', '000000071711 0.206 246.3 25.2 622.7 320.0\n', '000000071711 0.193 297.5 53.0 638.8 326.1\n', '000000071711 0.188 3.4 14.3 642.3 335.0\n', '000000071711 0.177 11.8 -5.8 440.7 336.2\n', '000000071711 0.172 1.0 2.7 640.0 327.1\n', '000000189828 0.656 362.9 99.5 569.7 205.2\n', '000000099054 0.495 2.7 214.4 427.1 554.0\n', '000000099054 0.317 31.5 71.3 427.8 558.2\n', '000000099054 0.260 309.8 212.8 363.7 233.5\n', '000000099054 0.229 1.3 86.7 426.2 558.5\n', '000000099054 0.196 53.6 111.4 430.2 318.2\n', '000000099054 0.168 4.4 154.4 428.7 548.5\n', '000000099054 0.120 386.9 221.6 424.5 235.2\n', '000000099054 0.117 333.7 215.8 366.7 233.3\n', '000000099054 0.113 67.8 116.5 422.5 416.4\n', '000000099054 0.112 387.3 223.2 412.3 233.4\n', '000000110721 0.648 18.0 0.9 638.2 357.2\n', '000000110721 0.157 119.7 1.6 154.4 19.2\n', '000000110721 0.109 141.5 2.3 158.9 19.6\n', '000000110721 0.100 141.5 1.1 170.0 13.5\n', '000000110721 0.090 76.2 1.6 121.7 21.6\n', '000000110721 0.089 108.4 1.2 135.7 16.6\n', '000000338325 0.698 117.9 137.8 453.4 304.1\n', '000000208901 0.695 233.6 146.5 424.9 284.7\n', '000000110359 0.676 219.7 146.9 422.2 384.7\n', '000000001761 0.667 278.4 138.8 334.0 186.1\n', '000000001761 0.638 150.1 7.4 224.7 92.3\n', '000000090631 0.720 178.9 119.0 451.4 229.3\n', '000000090631 0.097 177.3 202.3 247.6 227.7\n', '000000553094 0.655 173.2 47.2 451.9 405.3\n', '000000502599 0.668 126.8 292.7 386.7 402.2\n', '000000502599 0.571 393.1 323.6 591.6 410.8\n', '000000502599 0.569 18.5 124.2 464.2 228.9\n', '000000502599 0.551 2.0 210.0 187.7 308.4\n', '000000502599 0.149 4.6 3.8 288.5 152.6\n', '000000502599 0.113 0.7 319.9 27.4 347.4\n', '000000502599 0.105 428.9 -3.7 640.6 418.3\n', '000000502599 0.104 7.0 7.8 318.5 159.7\n', '000000502599 0.095 226.5 121.5 318.3 144.6\n', '000000502599 0.087 6.9 19.9 416.2 157.1\n', '000000396903 0.689 25.9 274.8 405.1 402.3\n', '000000229747 0.693 173.4 191.6 422.8 345.1\n', '000000139871 0.725 18.6 95.1 624.7 310.9\n', '000000502347 0.633 269.2 85.5 297.0 98.5\n', '000000452122 0.714 76.9 126.5 553.2 275.5\n', '000000190676 0.622 8.1 67.7 215.9 145.7\n', '000000477441 0.726 95.2 154.7 410.9 292.6\n', '000000477441 0.134 76.6 135.1 616.6 296.8\n', '000000477441 0.098 307.6 183.8 346.6 228.3\n', '000000205401 0.183 157.0 138.4 537.6 271.3\n', '000000205401 0.085 110.0 189.9 168.6 213.8\n', '000000408112 0.729 0.0 104.6 412.5 323.8\n', '000000408112 0.680 415.7 168.6 626.0 233.2\n', '000000408112 0.207 623.4 163.5 641.1 207.2\n', '000000408112 0.186 623.9 161.2 641.2 201.2\n', '000000408112 0.163 407.4 174.6 575.5 232.9\n', '000000408112 0.132 472.6 165.4 623.1 214.5\n', '000000408112 0.123 415.7 186.8 500.7 227.6\n', '000000408112 0.098 264.8 204.5 315.9 215.1\n', '000000408112 0.096 262.0 204.1 330.4 213.6\n', '000000479912 0.672 -0.2 222.3 339.0 446.9\n', '000000479912 0.460 113.9 261.0 146.5 278.9\n', '000000479912 0.176 129.0 261.4 194.2 280.7\n', '000000479912 0.148 150.4 264.0 190.3 277.9\n', '000000479912 0.147 54.4 265.0 89.6 274.0\n', '000000479912 0.140 53.7 265.7 88.7 274.5\n', '000000479912 0.129 0.5 321.3 17.3 333.9\n', '000000479912 0.128 95.3 265.2 117.5 276.5\n', '000000479912 0.117 154.2 265.4 194.5 278.6\n', '000000479912 0.083 190.1 269.9 210.6 283.7\n', '000000479912 0.083 1.9 263.7 16.5 272.5\n', '000000052017 0.729 58.6 80.7 478.1 289.2\n', '000000163746 0.511 175.9 -0.2 230.2 61.3\n', '000000163746 0.455 520.8 -0.0 575.1 61.1\n', '000000033114 0.713 170.6 170.5 504.4 276.7\n', '000000013348 0.727 7.5 135.0 572.0 283.0\n', '000000013348 0.450 1.6 224.9 52.9 244.1\n', '000000013348 0.222 1.1 220.5 47.0 242.1\n', '000000543528 0.707 186.0 78.9 519.4 245.6\n', '000000167540 0.722 169.1 79.8 489.8 326.9\n', '000000384350 0.652 3.6 47.8 639.1 249.2\n', '000000384350 0.190 194.8 248.7 625.5 338.6\n', '000000384350 0.105 204.2 249.6 632.7 349.4\n', '000000384350 0.101 194.7 251.5 575.6 334.7\n', '000000384350 0.071 193.1 254.0 598.4 344.6\n', '000000044652 0.695 75.3 165.1 270.9 248.7\n', '000000044652 0.084 146.8 172.5 268.0 248.5\n', '000000044652 0.067 77.2 179.4 135.4 231.1\n', '000000410221 0.655 270.1 137.6 377.1 254.9\n', '000000478862 0.704 9.3 55.5 642.0 341.9\n', '000000579158 0.725 2.0 3.0 589.9 424.6\n', '000000579158 0.285 519.7 338.7 646.5 384.6\n', '000000579158 0.166 539.6 345.1 642.2 396.2\n', '000000579158 0.161 526.2 241.6 639.3 390.1\n', '000000579158 0.135 1.0 228.9 193.0 333.4\n', '000000579158 0.127 519.2 329.0 631.0 382.5\n', '000000579158 0.121 530.0 119.4 637.2 394.2\n', '000000579158 0.115 518.3 313.3 639.9 381.4\n', '000000579158 0.110 3.8 231.1 192.5 331.3\n', '000000579158 0.105 536.3 338.0 646.7 483.2\n', '000000214205 0.711 252.5 310.8 400.7 360.6\n', '000000379453 0.703 123.8 123.6 350.4 309.6\n', '000000144114 0.705 34.9 178.1 581.3 337.4\n', '000000144114 0.100 542.0 189.6 575.9 203.9\n', '000000520324 0.673 142.1 216.0 635.7 350.5\n', '000000520324 0.280 145.6 222.1 350.3 276.4\n', '000000520324 0.091 584.9 256.8 640.8 288.9\n', '000000520324 0.081 270.5 245.1 638.1 350.8\n', '000000520324 0.078 161.1 219.2 579.6 271.1\n', '000000520324 0.077 142.3 266.7 239.3 329.8\n', '000000098520 0.722 88.9 126.6 616.2 285.5\n', '000000504000 0.574 17.9 45.4 652.6 217.9\n', '000000504000 0.508 -1.4 48.6 94.6 139.4\n', '000000210502 0.090 0.6 26.7 639.6 451.1\n', '000000348881 0.362 1.4 42.4 38.2 72.0\n', '000000348881 0.335 13.0 48.8 160.2 231.0\n', '000000348881 0.276 6.8 24.8 83.3 57.5\n', '000000348881 0.229 0.4 25.4 44.9 58.3\n', '000000348881 0.186 0.5 25.7 80.1 71.9\n', '000000348881 0.173 11.4 1.2 156.6 228.5\n', '000000348881 0.165 120.1 46.9 283.6 208.1\n', '000000348881 0.144 13.1 23.0 279.3 222.4\n', '000000348881 0.131 15.2 38.4 262.4 240.2\n', '000000348881 0.121 -1.6 35.2 290.3 223.9\n', '000000348881 0.105 578.4 195.2 641.6 242.8\n', '000000571857 0.532 260.1 179.7 448.0 231.1\n', '000000571857 0.425 22.5 160.2 485.3 239.1\n', '000000571857 0.333 441.4 192.7 495.8 207.0\n', '000000571857 0.330 2.6 186.3 87.5 227.7\n', '000000571857 0.296 2.8 152.9 501.7 261.3\n', '000000571857 0.161 344.6 189.1 430.1 211.5\n', '000000571857 0.155 339.4 180.7 497.8 210.3\n', '000000571857 0.154 6.1 154.4 249.8 237.8\n', '000000571857 0.131 9.2 161.5 494.7 241.8\n', '000000571857 0.114 329.3 190.2 408.1 207.1\n', '000000571857 0.110 19.6 150.2 370.2 251.7\n', '000000571857 0.108 323.9 192.6 438.3 227.2\n', '000000571857 0.102 8.1 160.5 288.5 242.9\n', '000000571857 0.098 15.9 158.4 478.6 279.0\n', '000000571857 0.097 0.2 158.8 517.6 272.2\n', '000000571857 0.093 0.9 191.9 22.4 214.6\n', '000000571857 0.091 357.2 187.6 431.3 202.2\n', '000000571857 0.088 -1.8 154.8 480.0 262.6\n', '000000571857 0.084 -0.3 158.1 89.2 227.6\n', '000000571857 0.082 459.0 192.6 501.2 206.0\n', '000000571857 0.080 14.4 152.7 415.8 242.1\n', '000000571857 0.078 0.9 188.3 22.7 207.7\n', '000000571857 0.078 353.3 168.1 506.0 207.4\n', '000000183500 0.742 91.1 80.2 538.2 343.3\n', '000000485802 0.656 207.1 193.7 222.4 209.6\n', '000000485802 0.106 186.1 204.5 213.2 312.2\n', '000000495054 0.718 66.1 130.2 607.4 278.5\n', '000000269682 0.056 333.6 218.4 353.7 242.8\n', '000000271457 0.156 0.7 1.9 499.6 331.4\n', '000000271457 0.103 0.3 129.3 499.3 326.5\n', '000000271457 0.095 0.4 84.0 498.0 331.8\n', '000000271457 0.084 -0.8 111.7 497.9 328.9\n', '000000322829 0.116 82.6 50.2 528.2 304.6\n', '000000322829 0.066 85.1 52.2 535.7 369.7\n', '000000290619 0.074 447.0 274.3 488.1 312.2\n', '000000300659 0.666 203.6 193.5 234.7 240.1\n', '000000300659 0.647 255.5 178.8 285.0 227.2\n', '000000300659 0.640 312.1 221.9 340.5 270.2\n', '000000300659 0.621 308.8 164.4 338.0 212.4\n', '000000300659 0.617 306.5 280.9 335.8 330.8\n', '000000300659 0.609 229.4 61.0 255.7 110.0\n', '000000300659 0.541 270.2 114.2 296.7 163.5\n', '000000300659 0.090 303.7 288.8 331.7 319.7\n', '000000300659 0.086 86.3 51.4 233.0 112.1\n', '000000300659 0.078 302.6 286.5 329.8 328.0\n', '000000392481 0.703 339.2 204.1 466.7 299.7\n', '000000135872 0.119 1.1 293.3 151.0 366.4\n', '000000157847 0.215 374.4 95.1 402.5 114.1\n', '000000200667 0.132 0.4 2.1 639.4 422.4\n', '000000200667 0.088 349.3 4.3 618.7 326.3\n', '000000039484 0.267 606.0 13.9 633.5 23.2\n', '000000272049 0.577 1.1 34.6 205.6 108.6\n', '000000272049 0.285 18.1 105.3 246.2 240.4\n', '000000272049 0.137 19.8 131.1 156.3 240.1\n', '000000272049 0.090 16.6 35.2 230.2 110.0\n', '000000304365 0.081 191.8 218.8 209.7 226.7\n', '000000042178 0.089 34.3 205.3 638.8 474.0\n', '000000249643 0.113 580.7 178.4 640.3 274.5\n', '000000249643 0.091 503.1 185.7 586.9 267.4\n', '000000225946 0.080 185.8 157.3 507.1 274.4\n', '000000478286 0.138 201.8 81.6 251.0 102.9\n', '000000368038 0.066 82.7 18.7 633.9 290.5\n', '000000057244 0.171 0.3 39.5 639.2 424.7\n', '000000184400 0.084 390.3 92.8 434.6 112.5\n', '000000475365 0.084 3.5 1.3 461.5 423.3\n', '000000377882 0.109 474.6 125.4 643.0 277.7\n', '000000234660 0.243 237.2 104.5 643.5 467.9\n', '000000234660 0.204 33.1 94.9 626.7 316.7\n', '000000273617 0.081 474.0 52.9 525.4 105.5\n', '000000046872 0.094 -1.0 83.7 504.0 387.4\n', '000000205289 0.084 147.6 122.4 488.2 285.5\n', '000000333956 0.094 0.1 314.5 50.3 368.2\n', '000000301061 0.268 128.2 -6.1 479.0 631.9\n', '000000174004 0.082 17.5 105.3 616.3 367.0\n', '000000106281 0.129 155.3 42.5 232.0 64.2\n', '000000106281 0.089 141.5 40.3 243.8 73.6\n', '000000041635 0.206 371.8 86.2 425.2 101.1\n', '000000041635 0.167 372.3 89.7 381.4 100.2\n', '000000041635 0.146 411.4 87.6 425.5 99.2\n', '000000041635 0.118 462.8 86.8 470.3 96.6\n', '000000390826 0.076 190.2 138.8 331.7 376.8\n', '000000061418 0.208 158.4 138.5 248.5 175.7\n', '000000503823 0.082 418.2 125.1 464.9 154.3\n', '000000365098 0.057 40.4 100.7 635.7 403.0\n', '000000561335 0.099 207.4 76.5 385.6 322.3\n', '000000369037 0.100 -1.0 5.2 422.8 634.3\n', '000000519208 0.069 1.4 0.8 640.4 405.5\n', '000000308328 0.100 4.2 43.6 637.5 471.6\n', '000000308328 0.094 70.3 56.0 602.1 470.3\n', '000000341094 0.259 3.2 67.8 502.1 366.5\n', '000000341094 0.086 1.0 30.7 499.9 368.6\n', '000000307658 0.059 8.2 52.3 537.8 636.5\n', '000000079144 0.099 137.2 136.8 271.0 264.0\n', '000000079144 0.071 251.1 175.2 343.8 367.1\n', '000000085478 0.068 27.7 81.2 218.8 326.6\n', '000000308753 0.175 250.7 226.1 412.9 279.7\n', '000000308753 0.069 375.5 443.0 645.4 480.5\n', '000000194506 0.067 71.2 114.9 307.0 274.2\n', '000000194216 0.089 489.5 152.5 636.0 257.9\n', '000000359855 0.087 139.0 13.6 563.0 330.1\n', '000000226592 0.100 100.6 110.6 645.1 451.6\n', '000000172935 0.299 19.0 100.3 270.7 216.0\n', '000000172935 0.071 77.2 142.9 393.9 223.9\n', '000000125211 0.100 75.8 95.0 636.8 479.0\n', '000000215245 0.086 162.5 199.2 207.7 231.6\n', '000000327780 0.072 0.9 1.3 639.9 479.9\n', '000000558213 0.087 342.7 189.6 637.8 302.7\n', '000000435081 0.100 1.9 2.3 499.6 499.4\n', '000000161799 0.526 472.8 73.4 507.8 88.4\n', '000000185292 0.067 1.9 140.4 421.7 193.1\n', '000000331604 0.067 336.8 106.1 369.6 150.3\n', '000000130613 0.076 215.1 74.6 489.6 353.6\n', '000000468245 0.094 326.5 1.0 519.1 39.1\n', '000000160666 0.382 3.8 78.5 639.7 263.9\n', '000000160666 0.174 -5.3 83.0 633.6 388.4\n', '000000160666 0.110 0.8 75.6 639.7 439.7\n', '000000160666 0.078 119.9 86.4 642.6 262.3\n', '000000160666 0.078 28.6 90.5 637.0 364.7\n', '000000160666 0.068 -7.7 78.9 631.2 426.9\n', '000000128675 0.089 320.9 13.5 357.3 85.9\n', '000000322895 0.101 398.1 55.2 406.0 59.1\n', '000000322895 0.082 382.5 61.4 388.0 65.7\n', '000000400922 0.648 379.4 125.2 406.8 138.4\n', '000000140929 0.134 195.4 7.2 630.1 277.4\n', '000000140929 0.122 238.7 7.4 633.9 277.0\n', '000000140929 0.114 171.8 12.3 636.0 341.5\n', '000000140929 0.072 193.1 15.8 643.9 343.2\n', '000000140929 0.070 1.1 2.5 639.7 395.8\n', '000000236599 0.229 463.3 67.5 501.0 92.6\n', '000000236599 0.212 424.1 49.4 476.5 81.8\n', '000000007784 0.059 1.3 83.2 335.6 371.9\n', '000000186345 0.437 278.3 56.4 290.9 70.6\n', '000000186345 0.392 319.6 41.6 329.6 58.3\n', '000000186345 0.225 277.1 67.8 290.3 77.9\n', '000000186345 0.137 278.0 56.4 289.8 69.8\n', '000000186345 0.130 261.2 84.1 275.2 101.2\n', '000000207306 0.086 162.4 148.4 212.0 175.1\n', '000000137294 0.136 1.1 0.6 640.0 426.2\n', '000000137294 0.105 2.3 3.1 405.1 428.6\n', '000000322844 0.082 169.4 305.4 479.2 549.3\n', '000000546826 0.159 1.7 1.1 640.7 480.0\n', '000000263425 0.136 35.3 146.4 610.0 423.2\n', '000000263425 0.075 5.2 5.4 636.2 427.6\n', '000000263425 0.075 38.5 105.6 605.4 429.2\n', '000000263425 0.074 61.1 134.0 618.9 425.6\n']

In [156]:
len(l1)

501

In [157]:
len(l2)

501

In [158]:
l1==l2

True